In [1]:
print("hello")

hello


In [2]:
import time
from pyspark.sql import SparkSession
import psutil
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, MapType
import pandas as pd
from functools import reduce
from pyspark import SparkContext
from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import col
from pyspark import StorageLevel

In [3]:
spark = (
        SparkSession.builder
        .appName("EEG_Analysis_HPC")

        # Target ~12 executors across the cluster (safe parallelism)
        .config("spark.executor.instances", "12")
        .config("spark.executor.cores", "1")             # 1 core per executor
        .config("spark.executor.memory", "4g")           # 12 × 4g = 48 GB total
        .config("spark.driver.memory", "6g")             # leave room for driver ops

        # Parallelism and shuffle tuning
        .config("spark.default.parallelism", "12")
        .config("spark.sql.shuffle.partitions", "12")

        # UDF / pandas optimization
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .config("spark.sql.execution.arrow.maxRecordsPerBatch", "500")

        # JVM GC tuning
        .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:+UseStringDeduplication")

        # Optional: Raise broadcast size if EEG config or schema is large
        .config("spark.sql.autoBroadcastJoinThreshold", "20MB")

        .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/05 20:15:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/05 20:15:35 WARN SparkSession: Cannot use ndb.NDBSparkSessionExtension to configure session extensions.
java.lang.ClassNotFoundException: ndb.NDBSparkSessionExtension
	at java.base/java.lang.ClassLoader.findClass(ClassLoader.java:724)
	at org.apache.spark.util.ParentClassLoader.findClass(ParentClassLoader.java:35)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:594)
	at org.apache.spark.util.ParentClassLoader.loadClass(ParentClassLoader.java:40)
	at org.apache.spark.util.ChildFirstURLClassLoader.loadClass(ChildFirstURLClassLoader.java:57)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:527)
	at java.base/java.lang.Class.forName0(Native Method)
	at java.base/java.lang.Class.forName(C

In [5]:
print("New Spark session created successfully")
print("=== Spark Configuration & Runtime info ===")
print(f"App ID: {spark.sparkContext.applicationId}")
print(f"Master: {spark.sparkContext.master}")
print(f"Default Parallelism: {spark.sparkContext.defaultParallelism}")
print(f"Total Executors: {spark.sparkContext._jsc.sc().getExecutorMemoryStatus().size()}")

for item in spark.sparkContext.getConf().getAll():
    print(f"{item[0]} = {item[1]}")
    

New Spark session created successfully
=== Spark Configuration & Runtime info ===
App ID: app-20250505201534-0000
Master: spark://cm012:38290
Default Parallelism: 12
Total Executors: 17
spark.driver.port = 35363
spark.default.parallelism = 12
spark.shuffle.io.connectionTimeout = 3000s
spark.sql.shuffle.partitions = 12
spark.ui.killEnabled = false
spark.ui.enabled = true
spark.driver.userClassPathFirst = true
spark.app.startTime = 1746490533383
spark.ndb.parallel_import = true
spark.ndb.access_key_id = 
spark.app.submitTime = 1746490533183
spark.ui.proxyBase = /rnode/cm012.hpc.nyu.edu/56849
spark.executor.userClassPathFirst = true
spark.rdd.compress = True
spark.executorEnv.JAVA_HOME = /usr/lib/jvm/java-11-openjdk-amd64
spark.master = spark://cm012:38290
spark.sql.catalog.ndb = spark.sql.catalog.ndb.VastCatalog
spark.ndb.retry_max_count = 3
spark.ndb.secret_access_key = 
spark.sql.execution.arrow.pyspark.enabled = true
spark.ndb.num_of_splits = 8
spark.rpc.io.connectionTimeout = 3000s
s

In [6]:
spark.stop()

25/05/05 20:15:48 WARN Dispatcher: Message RemoteProcessDisconnected(10.32.35.26:37420) dropped. Could not find BlockManagerEndpoint1.
25/05/05 20:15:48 WARN Dispatcher: Message RemoteProcessDisconnected(10.32.35.26:37402) dropped. Could not find BlockManagerEndpoint1.
25/05/05 20:15:48 WARN Dispatcher: Message RemoteProcessDisconnected(10.32.35.26:37460) dropped. Could not find BlockManagerEndpoint1.
25/05/05 20:15:48 WARN Dispatcher: Message RemoteProcessDisconnected(10.32.35.25:33088) dropped. Could not find BlockManagerEndpoint1.


In [ ]:
spark = (
    SparkSession.builder
    .appName("EEG_Analysis_HPC")
    .config("spark.executor.cores", "5")
    .config("spark.executor.memory", "4g")
    .config("spark.driver.memory", "6g")
    .getOrCreate()
)

In [ ]:
spark = (
        SparkSession.builder
        .appName("EEG_Analysis_HPC")

        # Target ~12 executors across the cluster (safe parallelism)
        .config("spark.executor.instances", "12")
        .config("spark.executor.cores", "1")             # 1 core per executor
        .config("spark.executor.memory", "4g")           # 12 × 4g = 48 GB total
        .config("spark.driver.memory", "6g")             # leave room for driver ops

        # Parallelism and shuffle tuning
        .config("spark.default.parallelism", "12")
        .config("spark.sql.shuffle.partitions", "12")

        # UDF / pandas optimization
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .config("spark.sql.execution.arrow.maxRecordsPerBatch", "500")

        # JVM GC tuning
        .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:+UseStringDeduplication")

        # Optional: Raise broadcast size if EEG config or schema is large
        .config("spark.sql.autoBroadcastJoinThreshold", "20MB")

        .getOrCreate()
    )